In [1]:
# Some IPython magic that allows us to ignore exceptions in the cells

from IPython.core.magic import register_cell_magic
import traceback
@register_cell_magic
def ignore_exceptions(line, cell):
    try:
        return exec(cell)
    except Exception as e:
        traceback.print_exc()

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
%config InlineBackend.figure_format='retina'

## Models of neural systems  <a class="tocSkip">

# NumPy: creating and manipulating numerical data

*Rike-Benjamin Schuppner, Institute for Theoretical Biology*

rike.schuppner@bccn-berlin.de

(See Ch. 3, **"Python Scientific lecture notes"**, Release 2012.3 (EuroScipy 2012) 

by the EuroScipy tutorial team

Editors: *Valentin Haenel, Emmanuelle Gouillart, GaÃ«l Varoquaux*

http://scipy-lectures.github.com) 

http://www.labri.fr/perso/nrougier/from-python-to-numpy/

Advanced Numpy patterns, *Juan Nunez-Iglesias*

### Copies and views (reminder)

* A slicing operation creates a **view** on the original array. 

* This is just a way of *accessing* array data. Thus the
original array is **not copied in memory**.

* When modifying the view, the original array is modified as well:

In [3]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
b = a[::2]
b

array([0, 2, 4, 6, 8])

In [5]:
b[0] = 12
b

array([12,  2,  4,  6,  8])

In [6]:
a # (!)

array([12,  1,  2,  3,  4,  5,  6,  7,  8,  9])

In [7]:
a = np.arange(10)
b = a[::2].copy() # force a copy
b[0] = 12
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

* This behavior can be surprising at first sight... 

* but it allows to save both memory and time.

**Warning:** *In-place* operators such as `=+` operate on views, explicit variants donâ€™t.

In [8]:
a = np.arange(3)
b = a
a += 1

print(f"In-place addition: {a=}, {b=}")

In-place addition: a=array([1, 2, 3]), b=array([1, 2, 3])


In [9]:
a = np.arange(3)
b = a
a = a + 1

print(f"Explicit addition: {a=}, {b=}")

Explicit addition: a=array([1, 2, 3]), b=array([0, 1, 2])


### Adding Axes

Indexing with the np.newaxis object allows us to add an axis to an array:

In [10]:
z = np.array([1, 2, 3])
z

array([1, 2, 3])

In [11]:
z[:, np.newaxis]

array([[1],
       [2],
       [3]])

In [12]:
z[np.newaxis, :]

array([[1, 2, 3]])

(`np.newaxis` is the same as `None` but for clarity you should always use `np.newaxis`.)

### Fancy indexing

* Numpy arrays can be indexed with *slices*

* but also with *boolean or integer arrays (masks)*

* This method is called fancy indexing. 

* It creates copies not views.

**Using boolean masks**

In [13]:
np.random.seed(3)
a = np.random.randint(0, 20, 15)
a

array([10,  3,  8,  0, 19, 10, 11,  9, 10,  6,  0, 12,  7, 14, 17])

In [14]:
a % 3 == 0

array([False,  True, False,  True, False, False, False,  True, False,
        True,  True,  True, False, False, False])

In [15]:
mask = (a % 3 == 0)
extract_from_a = a[mask] # or, a[a%3==0]
extract_from_a # extract a sub-array with the mask

array([ 3,  0,  9,  6,  0, 12])

* Indexing with a mask can be very useful to assign a new value to a sub-array:

In [16]:
a[a % 3 == 0] = -1
a

array([10, -1,  8, -1, 19, 10, 11, -1, 10, -1, -1, -1,  7, 14, 17])

**Indexing with an array of integers**

In [17]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

* Indexing can be done with an array of integers, where the same index is repeated several time:

In [18]:
a[ [2, 3, 2, 4, 2] ] # note: [2, 3, 2, 4, 2] is a Python list

array([2, 3, 2, 4, 2])

* New values can be assigned with this kind of indexing:

In [19]:
a[ [9, 7] ] = -10
a

array([  0,   1,   2,   3,   4,   5,   6, -10,   8, -10])

* When a new array is created by indexing with an array of integers, the new array has the same shape as the array
of integers:

In [20]:
a = np.arange(10)
idx = np.array(
    [[3, 4],
     [9, 7]]
)
a[idx]

array([[3, 4],
       [9, 7]])

**Consider the following examples of fancy indexing:**

In [21]:
a = np.arange(36).reshape(6,6)
a

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35]])

In [22]:
a[(0,1,2,3,4),(1,2,3,4,5)]

array([ 1,  8, 15, 22, 29])

In [23]:
a[:3,[0,2,5]]

array([[ 0,  2,  5],
       [ 6,  8, 11],
       [12, 14, 17]])

In [24]:
mask = np.array([1,0,1,0,0,1],dtype=bool)
a[mask,2]

array([ 2, 14, 32])

* We can even use *fancy indexing* and *broadcasting* (later) at the same time:

In [25]:
a = np.arange(12).reshape(3,4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [26]:
i = np.array([[0, 1], [1, 2]])
a[i, 2] # same as a[i, 2*np.ones((2, 2), dtype=int)]

array([[ 2,  6],
       [ 6, 10]])

Overview:
![](numpy_fancy_indexing.png)

### Numerical operations on arrays

Most of numpy and scipyâ€™s mathematical operations are *vectorized*. Applying these operations to arrays means they are applied elementwise.
However, similar to MATLAB, internally the elementwise operations are implemented in *C* and *C++*. Thus *vectorized* computations are much faster than if you would simply loop through each element yourself and apply your operation manually.

So alway try think *vectorized* instead of manually looping through arrays.

**Elementwise operations**

* With scalars:

In [27]:
a = np.array([1, 2, 3, 4])
a + 1

array([2, 3, 4, 5])

In [28]:
2**a

array([ 2,  4,  8, 16])

In [29]:
"""Speed up of vectorization"""
def twice_each_element(a):
    b=np.zeros(a.shape)
    
    for idx, elem in enumerate(a):
        b[idx] = 2*elem
    
    return b

a = np.arange(100)

%timeit twice_each_element(a)
%timeit 2*a

30.8 µs ± 47.1 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
982 ns ± 6.15 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


* All arithmetic operates elementwise:

In [30]:
a = np.array([1, 2, 3, 4])
b = np.ones(4) + 1
a - b

array([-1.,  0.,  1.,  2.])

In [31]:
a * b

array([2., 4., 6., 8.])

In [32]:
j = np.arange(5)
2**(j + 1) - j

array([ 2,  3,  6, 13, 28])

**Warning:** 

* Array multiplication is not matrix multiplication:

In [33]:
c = np.ones((3, 3))
c * c # NOT matrix multiplication!

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

* *Note:* Matrix multiplication:

In [34]:
np.matmul(c, c)

array([[3., 3., 3.],
       [3., 3., 3.],
       [3., 3., 3.]])

In [35]:
c @ c    # alternative syntax

array([[3., 3., 3.],
       [3., 3., 3.],
       [3., 3., 3.]])

* Comparisons:

In [36]:
a = np.array([1, 2, 3, 4])
b = np.array([4, 2, 2, 4])
a == b

array([False,  True, False,  True])

In [37]:
a > b

array([False, False,  True, False])

* Logical operations:

In [38]:
a = np.array([1, 1, 0, 0], dtype=bool)
b = np.array([1, 0, 1, 0], dtype=bool)
np.logical_or(a, b)

array([ True,  True,  True, False])

In [39]:
np.logical_and(a, b)

array([ True, False, False, False])

* Shape mismatches:

In [40]:
a = np.arange(4)
a

array([0, 1, 2, 3])

In [41]:
%%ignore_exceptions

a + np.array([1, 2])


Traceback (most recent call last):
  File "<ipython-input-1-2f68b7476f8f>", line 8, in ignore_exceptions
    return exec(cell)
  File "<string>", line 2, in <module>
ValueError: operands could not be broadcast together with shapes (4,) (2,) 


* **â€˜Broadcastâ€™?** Weâ€™ll return to that later.

* Transposition:

In [42]:
a = np.arange(8).reshape(2,4)
a

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

In [43]:
a.T

array([[0, 4],
       [1, 5],
       [2, 6],
       [3, 7]])

**Basic reductions**

* Computing sums:

In [44]:
x = np.array([1, 2, 3, 4])
np.sum(x)

10

In [45]:
x.sum()

10

* Sum by rows and by columns:

In [46]:
x = np.array([[1, 1], [2, 2]])
x

array([[1, 1],
       [2, 2]])

In [47]:
x.sum(axis=0) # columns (first dimension)

array([3, 3])

In [48]:
x[:, 0].sum(), x[:, 1].sum()

(3, 3)

In [49]:
x.sum(axis=1) # rows (second dimension)

array([2, 4])

In [50]:
x[0, :].sum(), x[1, :].sum()

(2, 4)

* Same idea in higher dimensions:

In [51]:
x = np.random.rand(3, 2, 4)
x_summed = x.sum(axis=1)
print(x_summed)

[[1.32755334 0.89499657 1.4660734  1.07218701]
 [1.40419069 0.5325054  1.34767287 1.47608727]
 [0.41960546 0.56741894 0.73902487 1.59606771]]


In [52]:
x[1, :, 3].sum()

1.4760872700564445

In [53]:
x_summed[1, 3]

1.4760872700564445

**Other reductionsâ€” works the same way (and take axis=)**

* Statistics:

In [54]:
x = np.array([1, 2, 3, 1])
y = np.array([[1, 2, 3], [5, 6, 1]])
x.mean()

1.75

In [55]:
np.median(x)

1.5

In [56]:
np.median(y, axis=-1) # last axis

array([2., 5.])

In [57]:
x.std() # full population standard dev.

0.82915619758885

* Extrema:

In [58]:
x = np.array([1, 3, 2])
x.min()

1

In [59]:
x.max()

3

In [60]:
x.argmin() # index of minimum

0

In [61]:
x.argmax() # index of maximum

1

* Logical operations:

In [62]:
np.all([True, True, False])

False

In [63]:
np.any([True, True, False])

True

**Note:** Can be used for array comparisons:

In [64]:
a = np.zeros((100, 100))
np.any(a != 0)

False

In [65]:
np.all(a == a)

True

In [66]:
a = np.array([1, 2, 3, 2])
b = np.array([2, 2, 3, 2])
c = np.array([6, 4, 4, 5])
((a <= b) & (b <= c)).all()

True

* ... and many more (best to learn as you go).

In [67]:
a = np.array([9,4,6,1,8])
print(f"{a=}")
print(f"minimum: {np.min(a)=}")
print(f"location of minimum: {np.argmin(a)=}")
print(f"check: {a[np.argmin(a)]=}")

a=array([9, 4, 6, 1, 8])
minimum: np.min(a)=1
location of minimum: np.argmin(a)=3
check: a[np.argmin(a)]=1


### Sorting data

* Sorting along an axis:

In [68]:
a = np.array([[4, 3, 5], [1, 2, 1]])
b = np.sort(a, axis=1)
b

array([[3, 4, 5],
       [1, 1, 2]])

* Note: Sorts each row separately!
    
* In-place sort:

In [69]:
a.sort(axis=1)
a

array([[3, 4, 5],
       [1, 1, 2]])

* Sorting with fancy indexing:

In [70]:
a = np.array([4, 3, 1, 2])
j = np.argsort(a)
j

array([2, 3, 1, 0])

In [71]:
a[j]

array([1, 2, 3, 4])

* Finding minima and maxima:

In [72]:
a = np.array([4, 3, 1, 2])
j_max = np.argmax(a)
j_min = np.argmin(a)
j_max, j_min

(0, 2)

### Exercise

Generate a 10 x 3 array of random numbers. From each row, pick the column containing the number closest to 0.75.

*Hint:* use of `np.abs` and `np.argmin` to find the column j that contains the closest element in each row i. The final result should be a vector of integers of shape (10,).

How do you use the result to select the minumum of each row?

In [73]:
rand_array = np.random.rand(10, 3)

# We use some freedom here to round our numbers, which makes printing nicer.
rand_array = np.round(rand_array, decimals=2)
rand_array

array([[0.71, 0.02, 0.32],
       [0.56, 0.75, 0.61],
       [0.2 , 0.65, 0.66],
       [0.87, 0.81, 0.37],
       [0.08, 0.76, 0.06],
       [0.84, 0.69, 0.45],
       [0.36, 0.52, 0.46],
       [0.81, 0.32, 0.84],
       [0.49, 0.54, 0.83],
       [0.52, 0.14, 0.3 ]])

The column indexes of each row that contain the number closest to 0.75:

In [74]:
indexes = np.argmin(np.abs(rand_array - 0.75), axis=1)
indexes

array([0, 1, 2, 1, 1, 1, 1, 0, 2, 0])

Using the `indexes` array to return the actual numbers that are closest to 0.75 turns out to be more tricky than expected.

If we try to select the rows with the slice index `:` (or similarly `0:10`), numpy gives us the following:

In [75]:
rand_array[:, indexes]

array([[0.71, 0.02, 0.32, 0.02, 0.02, 0.02, 0.02, 0.71, 0.32, 0.71],
       [0.56, 0.75, 0.61, 0.75, 0.75, 0.75, 0.75, 0.56, 0.61, 0.56],
       [0.2 , 0.65, 0.66, 0.65, 0.65, 0.65, 0.65, 0.2 , 0.66, 0.2 ],
       [0.87, 0.81, 0.37, 0.81, 0.81, 0.81, 0.81, 0.87, 0.37, 0.87],
       [0.08, 0.76, 0.06, 0.76, 0.76, 0.76, 0.76, 0.08, 0.06, 0.08],
       [0.84, 0.69, 0.45, 0.69, 0.69, 0.69, 0.69, 0.84, 0.45, 0.84],
       [0.36, 0.52, 0.46, 0.52, 0.52, 0.52, 0.52, 0.36, 0.46, 0.36],
       [0.81, 0.32, 0.84, 0.32, 0.32, 0.32, 0.32, 0.81, 0.84, 0.81],
       [0.49, 0.54, 0.83, 0.54, 0.54, 0.54, 0.54, 0.49, 0.83, 0.49],
       [0.52, 0.14, 0.3 , 0.14, 0.14, 0.14, 0.14, 0.52, 0.3 , 0.52]])

This actually applied the `indexes` array as a fancy index *for each individual row*. This is not what we wanted. (Note: We *could* run `np.diagonal` on this array as we are indeed only interested in the diagonal elements, but this seems more than a little wasteful on bigger arrays.)

What we want is to match row-idx 0 with col-idx `indexes[0]`, then row-idx 1 with col-idx `indexes[1]`. To make this explicit, we can do as follows:

In [76]:
rand_array[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], indexes]

array([0.71, 0.75, 0.66, 0.81, 0.76, 0.69, 0.52, 0.81, 0.83, 0.52])

Now, both elements are lists and numpy interprets it as we want it to.

Of course, we donâ€™t want enumerate all rows manually. Luckily, we can improve:

In [77]:
rand_array[range(len(indexes)), indexes]

array([0.71, 0.75, 0.66, 0.81, 0.76, 0.69, 0.52, 0.81, 0.83, 0.52])

## Broadcasting

* Basic operations on numpy arrays (addition, etc.) are elementwise
* This works on arrays of the same size.

**Nevertheless**

* Itâ€™s also possible to do operations on arrays of different sizes,
 if Numpy can transform these arrays so that they all have the same size

* this conversion is called broadcasting.

Example: We want to do multiply two vectors, each element with each other element:

In [78]:
a = np.arange(5)
b = np.logspace(0, 2, 3, dtype=int)
print(a); print(b)

[0 1 2 3 4]
[  1  10 100]


In [79]:
%%ignore_exceptions

a * b # wonâ€™t work

Traceback (most recent call last):
  File "<ipython-input-1-2f68b7476f8f>", line 8, in ignore_exceptions
    return exec(cell)
  File "<string>", line 2, in <module>
ValueError: operands could not be broadcast together with shapes (5,) (3,) 


Does not work with one-dimensional vectors. What if we work in two dimensions?

In [80]:
# Quick recap how numpy does dimensions

print(np.arange(4).reshape(4)); print()
print(np.arange(4).reshape(4, 1)); print()
print(np.arange(4).reshape(1, 4))

[0 1 2 3]

[[0]
 [1]
 [2]
 [3]]

[[0 1 2 3]]


Note: There is no difference in `ndim == 1` between row and column vectors. This exists only for two dimensions.

Now, letâ€™s assume a and b look as follows:

In [81]:
aMatrix = np.tile(a.reshape(5, 1), (1, 3))
bMatrix = np.tile(b.reshape(1, 3), (5, 1))

print(aMatrix); print(); print(bMatrix)
print(aMatrix.shape); aMatrix.shape == bMatrix.shape

[[0 0 0]
 [1 1 1]
 [2 2 2]
 [3 3 3]
 [4 4 4]]

[[  1  10 100]
 [  1  10 100]
 [  1  10 100]
 [  1  10 100]
 [  1  10 100]]
(5, 3)


True

Now we can multiply elementwise

In [82]:
(aMatrix * bMatrix)

array([[  0,   0,   0],
       [  1,  10, 100],
       [  2,  20, 200],
       [  3,  30, 300],
       [  4,  40, 400]])

In [83]:
(aMatrix * bMatrix)[2, 1]

20

In [84]:
a[2] * b[1]

20

In [85]:
for i, aa in np.ndenumerate(a):
    for j, bb in np.ndenumerate(b):
        assert a[i] * b[j] == (aMatrix * bMatrix)[i, j]

The good thing for us: Whenever numpy can tile automatically (and unambigously), it will do so. This is called broadcasting.

In [86]:
a.reshape(5, 1) * b.reshape(1, 3)

array([[  0,   0,   0],
       [  1,  10, 100],
       [  2,  20, 200],
       [  3,  30, 300],
       [  4,  40, 400]])

As we are only reshaping to add a new axis, we can be more explicit about this (and donâ€™t need to remember the exact shape):

In [87]:
a[:, np.newaxis]

array([[0],
       [1],
       [2],
       [3],
       [4]])

In [88]:
a[:, np.newaxis].shape

(5, 1)

In [89]:
a[:, np.newaxis] * b[np.newaxis, :]

array([[  0,   0,   0],
       [  1,  10, 100],
       [  2,  20, 200],
       [  3,  30, 300],
       [  4,  40, 400]])

Arrays in broadcasting are aligned starting from the last axis. Therefore we usually donâ€™t need to add axes to the beginning. This is equivalent:

In [90]:
a[:, np.newaxis] * b

array([[  0,   0,   0],
       [  1,  10, 100],
       [  2,  20, 200],
       [  3,  30, 300],
       [  4,  40, 400]])

### Broadcasting example

<table>
<thead>
<tr>
<th></th>
<th></th>
</tr>
</thead>
<tbody><tr>
<td>A</td>
<td>4d array</td>
</tr>
<tr>
<td>B</td>
<td>3d array</td>
</tr>
<tr>
<td>A + B</td>
<td>4d array</td>
</tr>
</tbody></table>


Broadcasting aligns the dimensions starting from the back.

In [91]:
A = np.arange(8*6).reshape(8, 1, 6, 1)
B = np.arange(7*5).reshape(   7, 1, 5)
(A * B).shape

(8, 7, 6, 5)

* We have already used broadcasting without knowing it!:

In [92]:
a = np.ones((4, 5))
a[0] = 2 # we assign an array of dimension 0 to an array of dimension 1
a

array([[2., 2., 2., 2., 2.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

* Broadcasting seems a bit magical
* but it is actually quite natural to use it
* e.g., when we want to solve a problem whose output data is an array with more dimensions than input data.

## Array shape manipulation

### Flattening

In [93]:
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6]])
a.ravel()

array([1, 2, 3, 4, 5, 6])

In [94]:
a.T.ravel()

array([1, 4, 2, 5, 3, 6])

* Higher dimensions: last dimensions ravel out â€œfirstâ€.
* Order can be changed with order={'C', 'F', 'A', 'K'}
* A copy is only done when needed (cf. `ndarray.flatten`)

### Reshaping

* The inverse operation to flattening:

In [95]:
a.shape

(2, 3)

In [96]:
b = a.ravel()
b.reshape((2, 3))

array([[1, 2, 3],
       [4, 5, 6]])

* Creating an array with a different shape, from another array:

In [97]:
a = np.arange(36)
b = a.reshape((6, 6))
b

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35]])

* Or,

In [98]:
b = a.reshape((6, -1)) # unspecified (-1) value is inferred
b

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35]])

### Views and copies

* ndarray.reshape may return a view (cf `help(np.reshape)`), not a copy:

In [99]:
b[0, 0] = 99
a

array([99,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35])

* Beware: reshape may also return a copy!

In [100]:
a = np.zeros((3, 2))
b = a.T.reshape(3*2)
b[0] = 9
a

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

* To understand, one would need to take a deeper look into the memory layout of an array.

### Dimension shuffling

In [101]:
a = np.arange(4*3*2).reshape(4, 3, 2)
a.shape

(4, 3, 2)

In [102]:
a[0, 2, 1]

5

In [103]:
b = a.transpose(1, 2, 0)
b.shape

(3, 2, 4)

In [104]:
b[2, 1, 0]

5

* Also creates a view:

In [105]:
b[2, 1, 0] = -1
a[0, 2, 1]

-1

### Resizing

* Size of an array can be changed with ndarray.resize:

In [106]:
a = np.arange(8)
a.resize((2,3,3))
a

array([[[0, 1, 2],
        [3, 4, 5],
        [6, 7, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]])

However, it must not be referred to somewhere else:

In [107]:
%%ignore_exceptions

b = a
a.resize((4,))

Traceback (most recent call last):
  File "<ipython-input-1-2f68b7476f8f>", line 8, in ignore_exceptions
    return exec(cell)
  File "<string>", line 3, in <module>
ValueError: cannot resize an array that references or is referenced
by another array in this way.
Use the np.resize function or refcheck=False
